# Part 1

/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

```SQL
SELECT name FROM `Facilities`
WHERE membercost <> 0;



/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT(name) FROM `Facilities`
WHERE membercost <> 0;


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */


SELECT facid, name, membercost FROM `Facilities`
WHERE (0 < membercost) AND   (membercost< 0.2 * monthlymaintenance);


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT * FROM `Facilities`
WHERE facid IN (1,5);


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance FROM `Facilities`
WHERE expense_label = 'cheap';

SELECT name, monthlymaintenance FROM `Facilities`
WHERE expense_label = 'expensive';



/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname FROM `Members`
WHERE joindate IN (SELECT MAX(joindate) FROM `Members`);


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT CONCAT(Members.surname, '-',  Facilities.name)
FROM `Bookings`
LEFT JOIN `Members` ON Bookings.memid = Members.memid
LEFT JOIN `Facilities` ON Bookings.facid = Facilities.facid 
WHERE Facilities.name IN ('Tennis Court 1', 'Tennis Court 2');


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT DISTINCT CONCAT(Members.surname, '-',  Facilities.name)
FROM `Bookings`
LEFT JOIN `Members` ON Bookings.memid = Members.memid
LEFT JOIN `Facilities` ON Bookings.facid = Facilities.facid 
WHERE (Bookings.starttime LIKE '2012-09-14%') AND (Facilities.membercost > 30)
ORDER BY Facilities.membercost DESC;


/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT DISTINCT CONCAT(Members.surname, '-',  Facilities.name)
FROM `Bookings`
LEFT JOIN `Members` ON Bookings.memid = Members.memid
LEFT JOIN `Facilities` ON Bookings.facid = Facilities.facid 
WHERE Bookings.starttime = (SELECT starttime FROM `Bookings` WHERE starttime LIKE '2012-09-14%') AND (Facilities.membercost > 30)
ORDER BY Facilities.membercost DESC;



In [12]:
import pandas as pd 
import pandasql as psql

Bookings = pd.read_csv("Bookings.csv")
Facilities = pd.read_csv('Facilities.csv')
Members = pd.read_csv('Members.csv')


# Part 2

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [30]:

query = """
SELECT name, SUM(membercost)+SUM(guestcost) AS revenue
FROM Facilities
GROUP BY name
ORDER BY revenue;
"""


result10 = psql.sqldf(query, locals())

result10

,name,rev
0,Pool Table,5.0
1,Snooker Table,5.0
2,Table Tennis,5.0
3,Badminton Court,15.5
4,Squash Court,21.0
5,Tennis Court 1,30.0
6,Tennis Court 2,30.0
7,Massage Room 1,89.9
8,Massage Room 2,89.9


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [37]:
query11 = """
SELECT surname, firstname, recommendedby
FROM Members
ORDER BY surname, firstname;
"""

psql.sqldf(query11, locals())

,surname,firstname,recommendedby
0,Bader,Florence,9.0
1,Baker,Anne,9.0
2,Baker,Timothy,13.0
3,Boothe,Tim,3.0
4,Butters,Gerald,1.0
5,Coplin,Joan,16.0
6,Crumpet,Erica,2.0
7,Dare,Nancy,4.0
8,Farrell,David,NaN
9,Farrell,Jemima,NaN


/* Q12: Find the facilities with their usage by member, but not guests */

In [42]:
query="""
SELECT Facilities.name, Members.surname, Members.firstname
FROM Bookings 
LEFT JOIN Facilities ON Facilities.facid = Bookings.facid
LEFT JOIN Members ON Bookings.memid = Members.memid
WHERE Members.surname<> 'GUEST'
"""

psql.sqldf(query, locals())

,name,surname,firstname
0,Tennis Court 2,Smith,Darren
1,Tennis Court 2,Smith,Darren
2,Tennis Court 2,Smith,Darren
3,Tennis Court 2,Smith,Darren
4,Tennis Court 2,Smith,Darren
...,...,...,...
3155,Badminton Court,Crumpet,Erica
3156,Table Tennis,Crumpet,Erica
3157,Table Tennis,Crumpet,Erica
3158,Massage Room 1,Crumpet,Erica


/* Q13: Find the facilities usage by month, but not guests */

In [75]:
query = """
SELECT Facilities.name, strftime('%Y-%m', Bookings.starttime) AS yearmonth
FROM Bookings 
LEFT JOIN Facilities ON Facilities.facid = Bookings.facid
LEFT JOIN Members ON Bookings.memid = Members.memid
WHERE Members.surname <> 'GUEST'
GROUP BY Facilities.name, yearmonth
"""

psql.sqldf(query, locals())


,name,yearmonth
0,Badminton Court,2012-07
1,Badminton Court,2012-08
2,Badminton Court,2012-09
3,Massage Room 1,2012-07
4,Massage Room 1,2012-08
5,Massage Room 1,2012-09
6,Massage Room 2,2012-07
7,Massage Room 2,2012-08
8,Massage Room 2,2012-09
9,Pool Table,2012-07
